函式庫 宣告

In [93]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import time

讀取資料

In [94]:
Data = pd.read_excel('交易資料集(2).xlsx')

In [95]:
# 資料前處理：重新組織數據
# data = Data.groupby('INVOICE_NO')['ITEM_NO'].apply(set).tolist()
data = Data.groupby('INVOICE_NO')['ITEM_NO'].apply(lambda items: [str(item) for item in set(items)]).tolist()

In [96]:
# 轉換資料為適合進行關聯分析的格式
te = TransactionEncoder()
te_array = te.fit(data).transform(data)
df = pd.DataFrame(te_array, columns=te.columns_)

In [97]:
def Association_rule(Type, support_threshold, confidence_threshold):
    start = time.time();
    if Type == 'Apriori':
        # Apriori 演算法 計算
        frequent_itemsets = apriori(df, min_support=support_threshold, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_threshold)
    elif Type == 'FPgrowth':
        # FP-Growth 演算法 計算
        frequent_itemsets = fpgrowth(df, min_support=support_threshold, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_threshold)
    end = time.time();

    rules.to_excel(f'關聯性規則/{Type}_rules.xlsx', index=False)
    print("執行時間：%f 秒" % (end - start))
    return rules,(end - start)

In [98]:
def recommend_products(rules, antecedents):
    # 找出所有包含前項的規則
    relevant_rules = rules[rules['antecedents'].apply(lambda x: set(antecedents).issubset(set(x)))]
    # 從規則中提取推薦產品 後項
    recommended_products = set()
    for consequents in relevant_rules['consequents']:
        recommended_products.update(consequents)
    # 移除輸入的產品 確保推薦列表中不包含輸入的產品
    recommended_products.difference_update(antecedents)
    return recommended_products

Apriori 演算法

In [99]:
rules_apriori,t = Association_rule('Apriori', 0.001, 0.01)

執行時間：11.466935 秒


In [100]:
input_product = ['BAS316,115', 'BAV99,215']
recommendations_apriori = recommend_products(rules_apriori, input_product)
print("Apriori 推薦的產品:", recommendations_apriori)

Apriori 推薦的產品: {'2N7002BK,215', 'BAW56,215', 'BAV70,215', 'PDZ15B,115'}


FP-Growth 演算法

In [101]:
rules_fpgrowth,t = Association_rule('FPgrowth', 0.001, 0.01)

執行時間：0.736440 秒


In [102]:
input_product = ['BAS316,115', 'BAV99,215']
recommendations_fpgrowth = recommend_products(rules_fpgrowth, input_product)
print("FP-Growth 推薦的產品:", recommendations_fpgrowth)

FP-Growth 推薦的產品: {'2N7002BK,215', 'BAW56,215', 'BAV70,215', 'PDZ15B,115'}
